# TensorFlow Tutorial
[Tutorial](https://www.tensorflow.org/tutorials/quickstart/beginner)

## 1. Set up TensorFlow
Check if it's using GPU or CPU.

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  1


## 2. Load a dataset
Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0.

This also converts the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 3. Build a machine learning model

Build a `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

`Sequential` is useful for stacking layers where each layer has one input `tensor` and one output tensor. 
Layers are functions with a known mathematical structure that can be reused and have trainable variables. 
Most TensorFlow models are composed of layers. This model uses the `Flatten`, `Dense`, and `Dropout` layers.

For each example, the model returns a vector of **logits** or **log-odds** scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.57301325, -0.74390036, -0.37216696, -0.5810667 ,  0.45771423,
        -0.82462615, -0.02980839,  0.1465517 ,  0.23420456,  1.1779552 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to _probabilities_ for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.05150689, 0.04341602, 0.06296391, 0.05109375, 0.14437918,
        0.04004896, 0.0886699 , 0.10577144, 0.11546107, 0.29668885]],
      dtype=float32)

  Note: It is possible to bake the __tf.nn.softmax__ function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using losses.SparseCategoricalCrossentropy:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. 
This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(3.2176528)

Before you start training, configure and compile the model using Keras `Model.compile`. 
Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model
Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8592 - loss: 0.4829 
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9560 - loss: 0.1495
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9667 - loss: 0.1109
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9729 - loss: 0.0855
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9758 - loss: 0.0766


The `Model.evaluate` method checks the model's performance, usually on a `validation set` or `test set`.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 5ms/step - accuracy: 0.9766 - loss: 0.0752


[0.0751836895942688, 0.9765999913215637]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.1372850e-08, 1.4333100e-09, 5.7370285e-06, 2.5904415e-05,
        2.4280000e-11, 7.1414159e-09, 1.2791366e-15, 9.9996567e-01,
        1.6095250e-07, 2.4618687e-06],
       [1.3120092e-08, 3.3387962e-06, 9.9996686e-01, 2.8204093e-05,
        4.9323986e-17, 1.2828248e-06, 1.1766929e-07, 4.4181686e-14,
        1.3046110e-07, 3.7538728e-14],
       [2.6407400e-07, 9.9763298e-01, 7.0016875e-05, 4.0894654e-05,
        1.2793127e-05, 4.7679649e-05, 2.5223508e-05, 1.9067433e-03,
        2.6283573e-04, 6.1004721e-07],
       [9.9985003e-01, 8.4226146e-09, 9.7979602e-05, 1.7643434e-07,
        4.4009045e-08, 2.4485769e-05, 1.8970877e-05, 5.1237390e-07,
        3.7211475e-08, 7.7306668e-06],
       [8.6292375e-06, 5.9997791e-11, 2.6453090e-06, 2.2828803e-09,
        9.9903464e-01, 5.9454951e-08, 5.1710508e-06, 6.9353018e-06,
        2.0509464e-07, 9.4169605e-04]], dtype=float32)>

## Conclusion
Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.